**This notebook is created for training the data processed in Exploratory Data Analysis Notebook**. 

**The Data that we have is already processed and feature engineering has been performed due to which it has additional new features which will be used for training the model**

**Importing required libraries**

In [68]:
#importing libraries

import os
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.metrics import mean_squared_error 
from numpy import math
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import pickle


**Loading processed data for training**

In [ ]:
# loading data
d = os.path.dirname(os.getcwd())
#path = d+"\\data\\processed\\processed_data.csv"
path = "/content/drive/MyDrive/Almabetter/Mini project 1/processed_data.csv"
tripdata = pd.read_csv(path)
tripdata.head()

,Unnamed: 0,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_weekday,pickup_month,pickup_year,pickup_hour,fare_amount,distance,direction
0,104406,-73.954347,40.763839,-73.950884,40.771308,1.0,6,9.0,2013.0,19.0,5.0,0.880232,359.537472
1,155552,-73.977028,40.743278,-73.986602,40.728143,1.0,4,3.0,2011.0,17.0,5.3,1.866275,179.371431
2,75475,-73.983017,40.781655,-73.976823,40.788423,3.0,2,1.0,2009.0,17.0,4.1,0.915594,359.086132
3,142151,-73.982161,40.757050,-73.959005,40.783415,1.0,4,6.0,2009.0,15.0,8.9,3.520959,359.123122
4,109426,-73.958803,40.815331,-73.961285,40.811988,1.0,1,3.0,2011.0,8.0,3.3,0.426391,179.257899


**Checking training data details**

In [ ]:
# selecting the columns/features which will be used

train = tripdata[['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude','pickup_weekday', 'pickup_hour','pickup_month','pickup_year','distance','direction','fare_amount']]


In [ ]:
# checking data again

train.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_weekday,pickup_hour,pickup_month,pickup_year,distance,direction,fare_amount
0,1.0,-73.954347,40.763839,-73.950884,40.771308,6,19.0,9.0,2013.0,0.880232,359.537472,5.0
1,1.0,-73.977028,40.743278,-73.986602,40.728143,4,17.0,3.0,2011.0,1.866275,179.371431,5.3
2,3.0,-73.983017,40.781655,-73.976823,40.788423,2,17.0,1.0,2009.0,0.915594,359.086132,4.1
3,1.0,-73.982161,40.757050,-73.959005,40.783415,4,15.0,6.0,2009.0,3.520959,359.123122,8.9
4,1.0,-73.958803,40.815331,-73.961285,40.811988,1,8.0,3.0,2011.0,0.426391,179.257899,3.3


**Loading test data**

In [ ]:
# loading test data

d = os.path.dirname(os.getcwd())
#path = d+"\\data\\processed\\test_data.csv"
testdata = pd.read_csv("/content/drive/MyDrive/Almabetter/Mini project 1/test_data.csv")
testdata.head()

,Unnamed: 0,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_weekday,pickup_month,pickup_year,pickup_hour,fare_amount,distance,direction
0,54458,-73.961287,40.771617,-73.974797,40.761457,1,3,11,2009,19,7.7,1.603380,178.674663
1,193387,-73.937060,40.854562,-73.984868,40.773842,2,6,2,2013,13,24.0,9.836146,179.408433
2,57401,-73.978800,40.762280,-73.973973,40.762740,1,5,12,2011,14,4.1,0.409743,349.652379
3,115324,-73.958191,40.760422,-73.861557,40.865349,1,0,2,2015,12,34.0,14.221987,359.077704
4,71684,-73.951845,40.778344,-73.958517,40.783291,1,1,11,2009,15,4.5,0.786243,1.346232


In [ ]:
# selecting columns 

test= testdata[['passenger_count','pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'pickup_weekday', 'pickup_hour','distance', 'direction','fare_amount']] 
test.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_weekday,pickup_hour,distance,direction,fare_amount
0,1,-73.961287,40.771617,-73.974797,40.761457,3,19,1.603380,178.674663,7.7
1,2,-73.937060,40.854562,-73.984868,40.773842,6,13,9.836146,179.408433,24.0
2,1,-73.978800,40.762280,-73.973973,40.762740,5,14,0.409743,349.652379,4.1
3,1,-73.958191,40.760422,-73.861557,40.865349,0,12,14.221987,359.077704,34.0
4,1,-73.951845,40.778344,-73.958517,40.783291,1,15,0.786243,1.346232,4.5


In [ ]:
# defining independent variables which will be used for training

independent_variables = [ 'pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude','passenger_count','distance','direction','pickup_hour','pickup_weekday']


**I have removed some of the columns like pickup month and pickup year. This was due to the exploratory data analysis that we performed in Exploratory Data Analysis notebook. We observed that month and year  do not show any definite pattern as such. Also, I am expecting that user will not be providing previous years for trip fare prediction.** 

In [ ]:
# function to print rmse as metric value for different models

RMSE_list = []
def print_metrics(actual, predicted,independent_variables):
  '''
     function to print rmse as metric value for different models

  '''
  print('RMSE is {}'.format(math.sqrt(mean_squared_error(actual, predicted))))
  RMSE_list.append(math.sqrt(mean_squared_error(actual, predicted)))
 

## **Baseline Model**

**Evaluation metric that will be used will be RMSE - Root Mean Squared Error**

In [ ]:
# calculating average trip fare amount as predicted value for baseline model

avg_fare=round(np.mean(train['fare_amount']),2)
baseline_pred=np.repeat(avg_fare,test['fare_amount'].shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, test['fare_amount']))
print("Baseline RMSE of Validation data :",baseline_rmse)

Baseline RMSE of Validation data : 10.05952986389026


**RMSE value comes out to be 10.05. Now we have baseline model to compare rmse values attained with the different models that we will use.**

**We will try to decrease rmse value attained by different models. It should be less than rmse value for baseline model.**

## **Decision Tree Regressor**

In [ ]:
# fitting data into decision Tree Regressor**

dtregressor = DecisionTreeRegressor(random_state=0)
dtregressor.fit(train[independent_variables],train['fare_amount'])
y_test_predicted = dtregressor.predict(test[independent_variables])
y_train_pred = dtregressor.predict(train[independent_variables])
                  

**RMSE Values for training and test data using Decsision Tree Regressor**

In [ ]:
# RMSE vale for training and testing data

print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 4.111632292751368e-16
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 7.044672668584396


**RMSE value is less than that observed in case of baseline model, which is good. But now we will see that if RMSE value can further be reduced.**

**We will perform hyper parameter tuning to the model**

In [ ]:
# hyper parameter tuning using grid search CV 

parameters={"splitter":["best","random"],
            "min_samples_leaf":[1,2,3,4,5,6,7],
           "min_samples_split":[2,3,4,5,6],
            "max_depth":[4,5,6]
           }

dtregressor = DecisionTreeRegressor(random_state=0)
tuning_model=GridSearchCV(dtregressor,param_grid=parameters,cv=3,verbose=3)
tuning_model.fit(train[independent_variables],train['fare_amount'])
y_test_predicted = tuning_model.predict(test[independent_variables])
y_train_pred = tuning_model.predict(train[independent_variables])

Fitting 3 folds for each of 210 candidates, totalling 630 fits
[CV 1/3] END max_depth=4, min_samples_leaf=1, min_samples_split=2, splitter=best;, score=0.827 total time=   0.4s
[CV 2/3] END max_depth=4, min_samples_leaf=1, min_samples_split=2, splitter=best;, score=0.826 total time=   0.4s
[CV 3/3] END max_depth=4, min_samples_leaf=1, min_samples_split=2, splitter=best;, score=0.830 total time=   0.4s
[CV 1/3] END max_depth=4, min_samples_leaf=1, min_samples_split=2, splitter=random;, score=0.728 total time=   0.0s
[CV 2/3] END max_depth=4, min_samples_leaf=1, min_samples_split=2, splitter=random;, score=0.213 total time=   0.0s
[CV 3/3] END max_depth=4, min_samples_leaf=1, min_samples_split=2, splitter=random;, score=0.167 total time=   0.0s
[CV 1/3] END max_depth=4, min_samples_leaf=1, min_samples_split=3, splitter=best;, score=0.827 total time=   0.4s
[CV 2/3] END max_depth=4, min_samples_leaf=1, min_samples_split=3, splitter=best;, score=0.826 total time=   0.3s
[CV 3/3] END max_de

In [ ]:
# parameters selected by model while performing hyper parameter tuning

tuning_model.best_params_

{'max_depth': 6,
 'min_samples_leaf': 2,
 'min_samples_split': 6,
 'splitter': 'best'}

In [ ]:
# RMSE value for training and test data after performing hyper parameter tuning to the decision tree regressor model

print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 3.567580504774292
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 5.711846245775705


**After hyper parameter tuning, RMSE comes out to be around 5.7 which is less than the normal decision tree regressor model. So hyper parameter tuning the model, helped in this case**

**Gradient Boost Regressor**

In [ ]:
# fitting training data into Random Forest Regressor

GB=GradientBoostingRegressor()
GB.fit(train[independent_variables],train['fare_amount'])


GradientBoostingRegressor()

**RMSE value for Gradient Boosting Regressor Model.**

In [ ]:
y_test_predicted = GB.predict(test[independent_variables])
y_train_pred = GB.predict(train[independent_variables])
print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 3.359158544049057
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 5.495650414451166


In [34]:
GB.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

**We can observe that it is performing better than decision tree regressor method. Even after hyper parameter tuning the decision tree regressor, the rmse value comes out to be litte less for gradient boosting regressor**

**Let us now try to see if hyper parameter tuning the Gradient Boosting regressor, helps in reducing the rmse value more or not**

In [71]:
# Hyper parameter tuning the Gradient Boosting regressor model

GBR=GradientBoostingRegressor()
search_grid={"alpha":[0.1,0.01],'learning_rate': [0.1, 0.01],'n_estimators':[70,80,90],'max_depth':[2]}
search=GridSearchCV(estimator=GBR,param_grid=search_grid,n_jobs=-1,cv=5)
search.fit(train[independent_variables],train['fare_amount'])

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.01], 'learning_rate': [0.1, 0.01],
                         'max_depth': [2], 'n_estimators': [70, 80, 90]})

In [72]:
# checking best parameters
search.best_params_

{'alpha': 0.01, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 90}

In [73]:
# Evaluation metric for tuned random forest regressor model

y_test_predicted = search.predict(test[independent_variables])
y_train_pred = search.predict(train[independent_variables])
print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 3.528865490918838
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 5.45495372980409


**After trying hyper parameter tuning on Gradient Boost Regressor, we got some fractional decrease in rmse value**

##### **Saving hypertuned models for further use**

In [74]:
# create an iterator object with write permission - model.pkl
# saving gradient boost regressor model

with open('gbr_model_pkl', 'wb') as files:
    pickle.dump(search, files)

In [75]:
# saving decision tree regressor model

with open('dtr_model_pkl', 'wb') as files:
    pickle.dump(tuning_model, files)